# Darbas su SQLAlchemy

Modulis SQLAlchemy susideda iš dviejų dalių:

    SQL Alchemy Core – įrankis, skirtas manipuliuoti, modifikuoti ir paleisti įvairias SQL užklausas. Labai panašiai, kaip ir SQLite3, tik dirba su visomis duomenų bazėmis: https://docs.sqlalchemy.org/en/13/core/engines.html

    SQL Alchemy ORM (ORM – Object Related Mapping) – įrankis, skirtas automatiškai susieti Python objektus su lentelėmis duomenų bazėje ir vykdyti įvairius veiksmus (CRUD), nenaudojant SQL užklausų

SQLAlchemy nėra vienintelis ORM funkcionalumą siūlantis modulis. Net ir Python pagrindu yra sukurta daugiau įrankių. Panašius modulius turi JAVA (Hibernate), kitos programavimo kalbos.

# SQL Alchemy ORM

In [2]:
import datetime
from sqlalchemy import Column, Integer, String, Float, DateTime, create_engine
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///data/projektai.db')
# declarative_base grazina klase, tai musu Base yra klase
Base = declarative_base()

In [3]:
# kuriam lentele per klase
class Projektas(Base):
    # čia nurodom lentelės pavadinimą
    __tablename__ = 'Projektas'
    # čia bus stulpelis
    id = Column(Integer, primary_key=True)
    # jei pavadinimas skiriasi nuo stulpelio pavadinimo, galima ji nusirodyti kabutese "pavadinimas"
    name = Column("pavadinimas", String)
    price = Column("kaina", Float)
    created_date = Column("sukurta", DateTime, default=datetime.datetime.utcnow)

    def __init__(self, name, price):
        self.name = name
        self.price = price

    def __repr__(self):
        return f"({self.id}, {self.name}, {self.price}, {self.created_date})"


In [4]:
# paleidziam
Base.metadata.create_all(engine)

# CRUD
* create
* read
* update
* delete

Kaip įrašyti, nuskaityti, atnaujinti, ištrinti duomenis SQLAlchemy lentelėje?

## Kaip sukurti ryšį su sukurta DB kitame faile:

In [5]:
from sqlalchemy.orm import sessionmaker
# cia Session padarem kaip klase
# Session = sessionmaker(bind=engine)
# session = Session()
session = sessionmaker(bind=engine)()

## Kaip įrašyti duomenis į lentelę:

In [6]:
# create
projektas1 = Projektas("Pirmas projektas", 20000)
session.add(projektas1)
session.commit()

In [6]:
projektas2 = Projektas("Antras projektas", 55000)
session.add(projektas2)
session.commit()

## Kaip gauti duomenis iš lentelės (cRud):

In [16]:
# read one
projektai = session.query(Projektas).get(1)
# projektai
projektai.name
# projektai.price
# type(projektai.price)

'Pirmas projektas'

In [18]:
# read many (cRud)
projektai = session.query(Projektas).all()
for projektas in projektai:
    print(projektas.name, projektas.price)

Pirmas projektas 20000.0
Antras projektas 55000.0


In [21]:
# search
search = session.query(Projektas).filter(Projektas.name.ilike("%as")).all()
search

[(1, Pirmas projektas, 20000.0, 2022-10-14 07:38:31.907104),
 (2, Antras projektas, 55000.0, 2022-10-14 07:40:14.603042)]

In [22]:
search = session.query(Projektas).filter(Projektas.name.ilike("pirmas%")).all()
search

[(1, Pirmas projektas, 20000.0, 2022-10-14 07:38:31.907104)]

In [23]:
search2 = session.query(Projektas).filter(Projektas.price > 25000).all()
search2

[(2, Antras projektas, 55000.0, 2022-10-14 07:40:14.603042)]

In [25]:
search3 = session.query(Projektas).filter(
    Projektas.price > 20000,
    Projektas.name.ilike("Antras%"),
).all()
search3

[(2, Antras projektas, 55000.0, 2022-10-14 07:40:14.603042)]

In [27]:
# Update (crUd)
atnaujinimas_projektas = session.query(Projektas).get(2)
atnaujinimas_projektas.price = 51300
session.commit()

In [28]:
# Delete (cruD)
trinamas = session.query(Projektas).filter(Projektas.id == 2).one()
session.delete(trinamas)
session.commit()
